# Preamble

importing

In [ ]:
import numpy as np
from math import sqrt

import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

from sklearn.datasets import make_blobs

from sklearn.inspection import DecisionBoundaryDisplay

from plot_utils import visualize_tree, grid_plot


Making test data

In [ ]:
centers=[[-5, 0], [5, 5], [5, -5]]
blobs = make_blobs(centers=centers, cluster_std= 2.5, n_samples=300, random_state=504)

X, y = blobs

colors = ListedColormap(["green", "yellow", "magenta"])

plt.scatter(X[:, 0], X[:, 1], c=y, cmap=colors)

# Fitting a decision tree

Creating and fitting on the data. I didn't create train and test sets. This is only for visualization.

In [ ]:

from sklearn.tree import DecisionTreeClassifier

tree_clf = DecisionTreeClassifier(random_state=504, min_impurity_decrease=.02)
tree_clf.fit(X, y)

In [ ]:
visualize_tree(tree_clf, ["X_1", "X_2"], ["Green", "Yellow", "Magenta"])

The position in the center is predicted as magenta. You can also see this by following the decision tree.

In [ ]:
tree_clf.predict([[0, 0]])

The decision tree is quite clean, but of course it can't be perfect. We can check the probabilities of the predictions. These are proportional to the number of samples of the category in the leaf.

In [ ]:
tree_clf.predict_proba([[0, 0]])

This plot shows how the decision tree bisects the sample space into rectangles. Note that all lines are horizontal or vertical.

In [ ]:
DecisionBoundaryDisplay.from_estimator(tree_clf, X, alpha=.2, cmap=colors, xlabel='X_1', ylabel='X_2')
plt.scatter(X[:, 0], X[:, 1], c=y, cmap=colors)


# A harder example

Let's try creating a harder example

In [ ]:
blobs = make_blobs(centers=[[-2.5, 0], [2.5, 4], [8.5, -1.5]], cluster_std= 2.5, n_samples=300, random_state=504)

X, y = blobs

colors = ListedColormap(["green", "yellow", "magenta"])

plt.scatter(X[:, 0], X[:, 1], c=y, cmap=colors)

In [ ]:

tree_clf = DecisionTreeClassifier(random_state=504, min_impurity_decrease=.02)
tree_clf.fit(X, y)

We can see that this is a lot more complex even without changing the hyperparameters. The bisection of the sample space is also a lot more complex and seems to have weird artifacts not in the data.

In [ ]:
visualize_tree(tree_clf, ["X_1", "X_2"], ["Green", "Yellow", "Magenta"])

In [ ]:
DecisionBoundaryDisplay.from_estimator(tree_clf, X, alpha=.2, cmap=colors, xlabel='X_1', ylabel='X_2')
plt.scatter(X[:, 0], X[:, 1], c=y, cmap=colors)


# Overfitting

Let's try tuning the hyperparameters to get a more precise result.

In [ ]:

tree_clf = DecisionTreeClassifier(random_state=504)
tree_clf.fit(X, y)

In [ ]:
visualize_tree(tree_clf, ["X_1", "X_2"], ["Green", "Yellow", "Magenta"])

In [ ]:
DecisionBoundaryDisplay.from_estimator(tree_clf, X, alpha=.2, cmap=colors, xlabel='X_1', ylabel='X_2')
plt.scatter(X[:, 0], X[:, 1], c=y, cmap=colors)


Even without a test set, we can clearly see the overfitting.

# Random forest

Let's create a random forest to tackle the problem. We use 15 trees in the forest, which is low, but good for visualization.

In [ ]:
from sklearn.ensemble import RandomForestClassifier

forest_clf = RandomForestClassifier(random_state=504, min_impurity_decrease=.02, n_estimators=15, max_features=2)
forest_clf.fit(X, y)

In [ ]:
trees = forest_clf.estimators_

def plot(ax, model, title):
    ax.set_title(title)
    DecisionBoundaryDisplay.from_estimator(model, X, ax=ax, alpha=.2, cmap=colors)
    ax.scatter(X[:, 0], X[:, 1], c=y, cmap=colors)

def tree_plot(ax, tree, index):
    plot(ax, tree, "tree {}".format(index + 1))

axes = grid_plot(trees, 4, 4, tree_plot)
plot(axes[3][3], forest_clf, "Forest")


Clearly, the forest is better than any individual attempt